In [1]:
import pandas as pd
import requests
import json
import time
from datetime import datetime
from pprint import pprint
from config import api_key

In [2]:
url = "https://api.themoviedb.org/3/"

In [3]:
popular_100_df = pd.read_csv("Resources/Most popular 100 movies.csv")

# Casting info in these movies
### Most prominent actor

In [4]:
# For every popular movie in the list, find the cast info for 5 most prominent characters

pop_100_cast = []
actor_info = []

timeout_count = 1

for i in range(len(popular_100_df["movie id"])):
    credits_url = url + "movie/" + str(popular_100_df["movie id"][i]) + "/credits?api_key=" + api_key

    casting_data = requests.get(credits_url).json()
    
    try:
        if casting_data["cast"][0]["gender"] == 1:
            gender = "Female"
        elif casting_data["cast"][0]["gender"] == 2:
            gender = "Male"
        else:
            gender = "Unknown"

        pop_100_cast.append({"movie id": popular_100_df["movie id"][i], 
                            "title": popular_100_df["title"][i],
                            "character": casting_data["cast"][0]["character"],
                            "actor id": casting_data["cast"][0]["id"],
                            "actor name": casting_data["cast"][0]["name"],
                            "gender": gender
                            })
        actor_info.append({"actor id": casting_data["cast"][0]["id"],
                            "actor name": casting_data["cast"][0]["name"],
                            "gender": gender
                            })
        
    except IndexError:
        print("Cast info not available for movie: " + popular_100_df["title"][i])
    
    timeout_count+=1
    if (timeout_count%1000) == 0:
        time.sleep(10)

Cast info not available for movie: The Seventh Brother
Cast info not available for movie: Urotsukidoji III: Return of the Overfiend
Cast info not available for movie: One Man Band


In [5]:
pop_100_cast_df = pd.DataFrame.from_dict(pop_100_cast)
pop_100_cast_df = pop_100_cast_df[pop_100_cast_df["gender"] != 'Unknown']
actor_info_df = pd.DataFrame.from_dict(actor_info)
actor_info_df = actor_info_df[actor_info_df["gender"] != 'Unknown']
pop_100_cast_df.head()

,actor id,actor name,character,gender,movie id,title
0,1109,Kevin Peter Hall,The Predator,Male,169,Predator 2
1,85,Johnny Depp,Edward Scissorhands,Male,162,Edward Scissorhands
2,62,Bruce Willis,John McClane,Male,1573,Die Hard 2
3,1204,Julia Roberts,Vivian Ward,Female,114,Pretty Woman
5,1158,Al Pacino,Don Michael Corleone,Male,242,The Godfather: Part III


In [6]:
pop_100_cast_df.to_csv("Resources/Cast_100_popular_movies.csv")